In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm, trange
import pickle
import multiprocessing as mp
from scipy.stats import shapiro, normaltest

In [2]:
data = pd.concat([
    pd.read_json('./data/train.statement.jsonl',lines=True),
     pd.read_json('./data/dev.statement.jsonl',lines=True),
     pd.read_json('./data/test.statement.jsonl',lines=True),
])

In [3]:
stems = data['question'].apply(lambda x: x['stem']).tolist()

In [4]:
with open('graph.pkl','rb') as f:
    graph_state = pickle.load(f)

In [5]:
concepts = list(graph_state['concepts_maper'].keys())
concepts = [x for x in concepts if type(x) != float]

In [6]:
pd.Series(list(''.join(stems))).value_counts()[::-1][:60]

“        1
á        1
)        1
(        1
%        1
:        1
X        2
6        2
!        2
Q        3
7        3
Z        4
;        5
8        6
/        7
2        7
9        8
3        9
4       11
5       12
"       29
K       29
1       32
V       36
R       54
0       59
U       66
-       68
O       70
G       71
N       76
F       84
L       90
Y       93
P      107
E      113
C      113
D      147
M      166
z      334
B      340
q      403
A      656
j      749
J      757
S      758
x      909
H      990
'     1502
I     1620
.     1762
T     1898
,     4077
v     4803
W     5688
k     5719
b     8555
?    10890
f    12271
p    12333
Name: count, dtype: int64

In [7]:
def prepare_text(s):
    s = s.lower().strip()
    s = s.replace(',',' ').replace('?',' ').replace('.',' ').replace('(',' ').replace(')',' ')
    s = s.replace('“',' ').replace('%',' ').replace('-',' ').replace("'",' ').replace('"',' ')
    s = s.replace('!',' ').replace('/',' ').replace(';',' ')
    for n in '0123456789':
        s = s.replace(n,' ')
    return s.split()

In [8]:
pool = mp.Pool(8)
prep_stems = pool.map(prepare_text,tqdm(stems))

  0%|          | 0/10962 [00:00<?, ?it/s]

In [9]:
#for st in tqdm(prep_stems):
#    batch = []
#    for cp in concepts:
#        if cp in st:
#            batch.append(cp)
#    concepts_per_st.append(batch)

def sample_batch(st):
    batch = []
    for cp in concepts:
        if cp in st:
            batch.append(cp)
    return batch

pool = mp.Pool(8)
concepts_per_st = pool.map(sample_batch, tqdm(prep_stems))

  0%|          | 0/10962 [00:00<?, ?it/s]

In [10]:
shapiro(pd.Series(concepts_per_st).apply(len).tolist())

/usr/local/lib/python3.11/dist-packages/scipy/stats/_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


ShapiroResult(statistic=0.9399198889732361, pvalue=0.0)

In [11]:
sr = pd.Series(np.concatenate(concepts_per_st))

In [12]:
srp = sr.value_counts() / sr.shape[0]

In [17]:
srp.iloc[1000:].sum()

0.15107838917009309

In [20]:
good_concepts = srp.iloc[1000:].index.tolist()

In [24]:
concepts_per_st = [[x for x in cp if x in good_concepts] for cp in concepts_per_st]

In [44]:
import pickle 
with open('./data/simple_fined_concepts.pkl','wb') as f:
    pickle.dump(concepts_per_st,f)

In [41]:
concepts_per_st

[['blow', 'efforts', 'punishing', 'sanctions'],
 ['sammy'],
 ['choker', 'boutique', 'jewelry', 'locate'],
 ['services', 'maps', 'google', 'gps', 'replaced'],
 ['cable', 'entertainment'],
 ['baggage', 'checked', 'heading', 'drawstring'],
 ['forgotten', 'mold', 'leftovers', 'gotten'],
 ['absorb', 'fountain', 'ink'],
 [],
 ['hobby', 'relaxed', 'bench', 'johnny'],
 ['aromas', 'rooms', 'drew'],
 ['marsh', 'cajun', 'peaking'],
 ['slowly', 'grill'],
 ['contracts'],
 ['pizzeria'],
 ['soccer'],
 [],
 ['foot',
  'accelerator',
  'pedals',
  'controller',
  'via',
  'steering',
  'controlled',
  'throttle'],
 ['constantly'],
 ['demand', 'exotic', 'snakes'],
 ['glare'],
 [],
 ['gated'],
 ['europe', 'canals'],
 [],
 ['lay'],
 ['seafood'],
 [],
 ['promised'],
 ['accidents'],
 [],
 ['thirsty', 'drought', 'searching'],
 ['season', 'successful', 'skiing'],
 ['distance', 'expert', 'alarm', 'sensor', 'installation'],
 ['rushed', 'rush'],
 ['jacks'],
 ['guard', 'hired', 'duties'],
 ['cops'],
 ['fiddle'],
